In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import MultiLabelBinarizer

#Load CSV file into DataFrame
csv_file = "ted_talks_en.csv"
fullDataSet = pd.read_csv(csv_file)

fullDataSet = fullDataSet.replace(",", "")

with open("file_path.txt", 'w') as file:
    file.write(fullDataSet.transcript[2])
print(fullDataSet.transcript[2])

fullDataSet['topics'] = fullDataSet['topics'].apply(eval)

x = fullDataSet[["title", "description","transcript"]]
y = fullDataSet["topics"]

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 42)


import re

def preprocess_text(text):

  text = text.lower()  # Lowercase
  text = re.sub(r"[^a-z0-9 ]", "", text)  # Remove non-alphanumeric characters
  tokens = text.split()  # Tokenize
  return tokens


from transformers import BertTokenizer, BertForSequenceClassification

model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(set(fullDataSet['topics'].sum())))  # Adjust num_labels based on unique topics

def prepare_data(data, label):

    tokens_list = []  # Initialize an empty list to hold tokens
    for element in data:
        preprocessed_text = preprocess_text(element)
        tokens_list.extend(preprocessed_text)  # Extend the list with tokens

    text = " ".join(tokens_list)  # Join the tokens with space

    encoded_data = tokenizer(text, padding="max_length", truncation=True, return_tensors="pt")
    input_ids = encoded_data['input_ids'].squeeze(0)
    attention_mask = encoded_data['attention_mask'].squeeze(0)

    position_ids = torch.arange(input_ids.size(0))

    return input_ids, attention_mask, position_ids, label


from transformers import AdamW
import torch
import torch.nn as nn

optimizer = AdamW(model.parameters())
loss_fn = nn.CrossEntropyLoss()

for epoch in range(3):  # Adjust number of epochs
  for i in range(len(x_train)):
    data = x_train.iloc[i]  # Get entire row (Series)
    label = y_train.iloc[i]
    input_ids, attention_mask, *other_args = prepare_data(data, label)  # Unpack based on position_ids requirement
    
    if tokenizer.requires_position_ids:
      outputs = model(input_ids, attention_mask=attention_mask, position_ids=other_args[0])  # Use position_ids if required
    else:
      outputs = model(input_ids, attention_mask=attention_mask)
    
    loss = loss_fn(outputs.logits, label)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    def prepare_data(data, label):
    """
    Prepares training data for BERT model.
    Args:
        data: A Pandas Series containing text features (title, description, transcript).
        label: A single topic label.
    Returns:
        A tuple containing tokenized inputs, attention mask, position IDs (if required), and label.
    """
    tokens_list = []  
    for element in data:
        preprocessed_text = preprocess_text(element)
        tokens_list.extend(preprocessed_text)  # Extend the list with tokens

    text = " ".join(tokens_list) 

    encoded_data = tokenizer(text, padding="max_length", truncation=True, return_tensors="pt")
    input_ids = encoded_data['input_ids'].squeeze(0)
    attention_mask = encoded_data['attention_mask'].squeeze(0)

    if hasattr(model.config, "position_embedding_type"):
        position_ids = torch.arange(input_ids.size(1))
        return input_ids, attention_mask, position_ids, label
    else:
        return input_ids, attention_mask, label


# Usage
for epoch in range(3):  
    for i in range(len(x_train)):
        data = x_train.iloc[i] 
        label = y_train.iloc[i]
        *args, label = prepare_data(data, label) 
        if len(args) == 3:  
            input_ids, attention_mask, position_ids = args
        else:
            input_ids, attention_mask = args

        outputs = model(input_ids.unsqueeze(0), attention_mask=attention_mask.unsqueeze(0))
        loss = loss_fn(outputs.logits.squeeze(0), label)  # Squeeze the logits tensor to match label shape
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


def prepare_data(data, label):
    tokens_list = [] 
    for element in data:
        preprocessed_text = preprocess_text(element)
        tokens_list.extend(preprocessed_text)  

    text = " ".join(tokens_list)

    encoded_data = tokenizer(text, padding="max_length", truncation=True, return_tensors="pt")
    input_ids = encoded_data['input_ids'].squeeze(0)
    attention_mask = encoded_data['attention_mask'].squeeze(0)

    if hasattr(model.config, "position_embedding_type"):
        max_length = input_ids.size(0)
        position_ids = torch.arange(max_length)
        return input_ids, attention_mask, position_ids, label
    else:
        return input_ids, attention_mask, label

import torch
from sklearn.preprocessing import LabelEncoder
import numpy as np

loss_fn = torch.nn.CrossEntropyLoss()

for epoch in range(3):  
    for i in range(len(x_train)):
        data = x_train.iloc[i] 
        label = y_train.iloc[i]
        *args, label = prepare_data(data, label) 
        if len(args) == 3:  
            input_ids, attention_mask, position_ids = args
        else:
            input_ids, attention_mask = args

        outputs = model(input_ids.unsqueeze(0), attention_mask=attention_mask.unsqueeze(0))
        
        label_1d = np.array(label).flatten()
        
        label_encoder = LabelEncoder()
        label_encoded = label_encoder.fit_transform(label_1d)
        label_tensor = torch.tensor(label_encoded)

        loss = loss_fn(outputs.logits.squeeze(0), label_tensor)  # Use label_tensor in the loss function
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


def predict_topics(text):
  text = preprocess_text(text)
  encoded_text, attention_mask = prepare_data([text])
  with torch.no_grad():
    outputs = model(encoded_text, attention_mask=attention_mask)
  predicted_label = torch.argmax(outputs.logits, dim=1).item()
  return fullDataSet['topics'].iloc[predicted_label]  # Assuming topics list is indexed the same as labels
